In [1]:
from src.system.certificate.RASM import ReachAvoidSuperMartingaleCertificate
from src.system.polyhorn_helper import CommunicationBridge
from system.toolIO import Parser

parser = Parser(
    "./kaushik_sample/actions.json",
    "./kaushik_sample/states.json",
    "./kaushik_sample/dynamics.json",
    "./kaushik_sample/noise.json",
    "./kaushik_sample/config.json",
)
# parser = Parser(
#     "./kaushik_sample_linear/actions.json",
#     "./kaushik_sample_linear/states.json",
#     "./kaushik_sample_linear/dynamics.json",
#     "./kaushik_sample_linear/noise.json",
#     "./kaushik_sample_linear/config.json",
# )
# parser = Parser(
#     "./djordje_sample/actions.json",
#     "./djordje_sample/states.json",
#     "./djordje_sample/dynamics.json",
#     "./djordje_sample/noise.json",
#     "./djordje_sample/config.json",
# )
# parser = Parser(
#     "./djordje_sample_2/actions.json",
#     "./djordje_sample_2/states.json",
#     "./djordje_sample_2/dynamics.json",
#     "./djordje_sample_2/noise.json",
#     "./djordje_sample_2/config.json",
# )
# parser = Parser(
#     "./safe_linear_sample/actions.json",
#     "./safe_linear_sample/states.json",
#     "./safe_linear_sample/dynamics.json",
#     "./safe_linear_sample/noise.json",
#     "./safe_linear_sample/config.json",
# )
tool_input = parser.parse()
print(tool_input)

ToolInput(state_space=Space(dimension=2, inequalities='0<=S1**2<=400and0<=S2**2<=400', listed_space_inequalities=None), action_policy=SystemControlPolicy(action_space=Space(dimension=2, inequalities='0<=A1<=1and0<=A2<=1', listed_space_inequalities=None), state_dimension=2, maximal_degree=5, transitions=[Equation(monomials=[]), Equation(monomials=[])], verification_mode=True, generated_constants=set(), constant_founded=False), disturbance=SystemStochasticNoise(dimension=2, distribution_name='normal', distribution_generator_parameters={'mean': [0, 0], 'std_dev': [0.1, 0.1], 'seed': 42}, noise_generators=NormalNoiseGenerator(mean=[0, 0], std_dev=[0.1, 0.1], dimension=2, seed=42)), dynamics=SystemDynamics(state_dimension=2, action_dimension=2, disturbance_dimension=2, system_transformations=[Equation(monomials=[Monomial(coefficient=1.00000000000000, variable_generators=('S1',), power=(1,)), Monomial(coefficient=0.100000000000000, variable_generators=('S2',), power=(1,))]), Equation(monomia

In [2]:
RASM = ReachAvoidSuperMartingaleCertificate(
    maximal_degree=tool_input.synthesis_config.maximal_polynomial_degree,
    state_dimension=tool_input.state_space.dimension,
)

In [3]:
RASM.define_constraints(tool_input)

Epsilon: 1e-14
Decrease Expectation Constraint: (-1.00000000000000e-14) + (-0.100000000000000 * S2 * V2) + (-0.210000000000000 * S2**2 * V3) + (-0.331000000000000 * S2**3 * V4) + (-0.464100000000000 * S2**4 * V5) + (-0.610510000000000 * S2**5 * V6) + (0.0100000000000000 * S1 * V7) + (-0.0780000000000001 * S1 * S2 * V8) + (-0.173700000000000 * S1 * S2**2 * V9) + (-0.277760000000000 * S1 * S2**3 * V10) + (-0.390895000000000 * S1 * S2**4 * V11) + (0.0199000000000000 * S1**2 * V12) + (-0.0563300000000000 * S1**2 * S2 * V13) + (-0.138126000000000 * S1**2 * S2**2 * V14) + (-0.225851000000000 * S1**2 * S2**3 * V15) + (0.0297010000000000 * S1**3 * V16) + (-0.0349856000000000 * S1**3 * S2 * V17) + (-0.103265900000000 * S1**3 * S2**2 * V18) + (0.0394039900000001 * S1**4 * V19) + (-0.0139624549999999 * S1**4 * S2 * V20) + (0.0490099501000000 * S1**5 * V21) + (1.00000000000000e-5 * S1**10 * S2**5 * V6) + (1.00000000000000e-6 * S1**10 * S2**5 * V11) + (1.00000000000000e-7 * S1**10 * S2**5 * V15) + 

In [4]:
print(RASM)

RASM Certificate: 
+	V(x) = (V1) + (S2 * V2) + (S2**2 * V3) + (S2**3 * V4) + (S2**4 * V5) + (S2**5 * V6) + (S1 * V7) + (S1 * S2
 	* V8) + (S1 * S2**2 * V9) + (S1 * S2**3 * V10) + (S1 * S2**4 * V11) + (S1**2 * V12) + (S1**2 * S2
 	* V13) + (S1**2 * S2**2 * V14) + (S1**2 * S2**3 * V15) + (S1**3 * V16) + (S1**3 * S2 * V17) +
 	(S1**3 * S2**2 * V18) + (S1**4 * V19) + (S1**4 * S2 * V20) + (S1**5 * V21) 
- Constraints:
+	(V1) + (S2 * V2) + (S2**2 * V3) + (S2**3 * V4) + (S2**4 * V5) + (S2**5 * V6) + (S1 * V7) + (S1 * S2
 	* V8) + (S1 * S2**2 * V9) + (S1 * S2**3 * V10) + (S1 * S2**4 * V11) + (S1**2 * V12) + (S1**2 * S2
 	* V13) + (S1**2 * S2**2 * V14) + (S1**2 * S2**3 * V15) + (S1**3 * V16) + (S1**3 * S2 * V17) +
 	(S1**3 * S2**2 * V18) + (S1**4 * V19) + (S1**4 * S2 * V20) + (S1**5 * V21) >= 0; forall (400) +
 	(-1 * S1**2) >= 0 and (400) + (-1 * S2**2) >= 0 and (S1**2) >= 0 and (S2**2) >= 0
+	(1) + (-1 * V1) + (-1 * S2 * V2) + (-1 * S2**2 * V3) + (-1 * S2**3 * V4) + (-1 * S2**4 * V5) + (-1 *


In [5]:
for i, constraint in enumerate(RASM.get_constraints(), start=1):
    print(f"+ Constraint {i}:")
    print(" "*4, constraint.to_polyhorn_preorder())
    print()

+ Constraint 1:
     (assert (forall ((S1 Real) (S2 Real)) (=> (and (and (and (>= (+ 400 (* -1 (* S1 S1))) 0) (>= (+ 400 (* -1 (* S2 S2))) 0)) (>= (* 1 (* S1 S1)) 0)) (>= (* 1 (* S2 S2)) 0)) (>= (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (* 1 V1) (* (* 1 S2) V2)) (* (* 1 (* S2 S2)) V3)) (* (* 1 (* S2 (* S2 S2))) V4)) (* (* 1 (* (* S2 S2) (* S2 S2))) V5)) (* (* 1 (* (* S2 S2) (* S2 (* S2 S2)))) V6)) (* (* 1 S1) V7)) (* (* (* 1 S1) S2) V8)) (* (* (* 1 S1) (* S2 S2)) V9)) (* (* (* 1 S1) (* S2 (* S2 S2))) V10)) (* (* (* 1 S1) (* (* S2 S2) (* S2 S2))) V11)) (* (* 1 (* S1 S1)) V12)) (* (* (* 1 (* S1 S1)) S2) V13)) (* (* (* 1 (* S1 S1)) (* S2 S2)) V14)) (* (* (* 1 (* S1 S1)) (* S2 (* S2 S2))) V15)) (* (* 1 (* S1 (* S1 S1))) V16)) (* (* (* 1 (* S1 (* S1 S1))) S2) V17)) (* (* (* 1 (* S1 (* S1 S1))) (* S2 S2)) V18)) (* (* 1 (* (* S1 S1) (* S1 S1))) V19)) (* (* (* 1 (* (* S1 S1) (* S1 S1))) S2) V20)) (* (* 1 (* (* S1 S1) (* S1 (* S1 S1)))) V21)) 0)) ))

+ Constraint 2:
     (asse

# Feed to Polyhorn

In [6]:
input_string = CommunicationBridge.get_input_string(
    tool_io=tool_input,
    certificate=RASM,
)
input_config = CommunicationBridge.get_input_config(
    tool_io=tool_input,
)

In [7]:
# print(input_string)

In [ ]:
CommunicationBridge.feed_to_polyhorn(
    config=input_config,
    input_string=input_string
)

Running solver...
